# Little Washita ParFlow-CLM with subsurface properties table 
This script is running exactle the same simulation as the [Little Washita Annoated ParFlow-CLM simulation script](https://github.com/hydroframe/parflow_python_shortcourse/blob/main/exercises/little_washita/LittleWashita_ParFlowCLM_AnnotatedExample%20copy.ipynb). The only difference here is that we are illustrating how to read in the subsurface properties from a table.  For more thorough annotations refer to the annoated input script. 


**Requriements** 
Before you can run this workbook you will need to make sure that you have parflow and pftools installed. Refer to the main readme of this repo for instructions on getting your modeling environment setup. if you haven't do so already.

**Simulation Inputs**
1. Solid file: used to define the shape of the domain 
2. Indicator file: used to define where the different geologic units are in the grid
3. Slope files: slope x and slope y files define the topographic slopes of each grid cell in the x and y directions
4. Initial pressure:  a pressure file created from a spinup run of the model (refer to other Little Washita exercises for spinup)

### 1. Import required libraries and functions 
This is the same as previous codes with one addition: now we are importing the `SubsurfacePropertiesBuilder` which will allow us to set our subsurface parameters from a csv table that we read in rather than within the python script. 

In [ ]:
import os
import numpy as np
from parflow import Run 
import shutil
from parflow.tools.fs import mkdir, cp, get_absolute_path, exists
from parflow.tools.settings import set_working_directory
from parflow.tools.builders import SubsurfacePropertiesBuilder

## 2. Setup your run directory and initialize your model object

In [ ]:
# Name your ParFLow run -- note that all of your output files will have this prefix
runname = 'LW_CLM_withTable'

# Create a directory in the outputs folder for this run
run_dir = get_absolute_path(f'outputs/{runname}')
mkdir(run_dir)
print(run_dir)

# create your Parflow model object. For starters we are just goin to set the file version and the run directory we'll add more later
# note that the model will run from the run_dir so all input files should be in the run dir or paths should be specified relative to run_dir
model = Run(runname, run_dir)
model.FileVersion = 4

#copy the model inputs for the simulation into the run directory
#NOTE: you dont have to copy everything into the run directory if you don't want, you can also point to input files in other directories in a simulation if you prefer
input_dir= os.path.join(os.getcwd(), 'model_inputs')
files=os.listdir(input_dir)
for fname in files:
    shutil.copy(os.path.join(input_dir,fname), run_dir)

## 3. Setup the computational grid

### 3.1 processor toppology and computational grid

In [ ]:
# Processor topology: This is the way that the problem will be split across processors if you want to run in parallel
# The domain is divided in x,y and z dimensions by P, Q and R. The total number of processors is P*Q*R.
model.Process.Topology.P = 1
model.Process.Topology.Q = 1
model.Process.Topology.R = 1

#Locate the origin in the domain.
model.ComputationalGrid.Lower.X = 0.0
model.ComputationalGrid.Lower.Y = 0.0
model.ComputationalGrid.Lower.Z = 0.0

# Define the size of each grid cell. The length units are the same as those on hydraulic conductivity, here that is meters.
model.ComputationalGrid.DX = 1000.0
model.ComputationalGrid.DY = 1000.0
model.ComputationalGrid.DZ = 200.0

# Define the number of grid blocks in the domain.
model.ComputationalGrid.NX = 64
model.ComputationalGrid.NY = 32
model.ComputationalGrid.NZ = 10

### 3.2 Geometries

In [ ]:
#Declare the geometries that you will use for the problem
model.GeomInput.Names = "solid_input indi_input"

#Define the solid_input geometry.  
#Note the naming convention here GeomInput.{GeomName}.key
model.GeomInput.solid_input.InputType = "SolidFile"
model.GeomInput.solid_input.GeomNames = "domain"
model.GeomInput.solid_input.FileName = "LW.pfsol"

#First set the name for your `Domain` and setup the patches for this domain
model.Domain.GeomName = "domain"
model.Geom.domain.Patches = "top bottom side"

#Indicator file geometry
model.GeomInput.indi_input.InputType =   "IndicatorField"
model.GeomInput.indi_input.GeomNames = "s1 s2 s3 s4 s5 s6 s7 s8 s9 s10 s11 s12 s13 g1 g2 g3 g4 g5 g6 g7 g8 g9 g10"
model.Geom.indi_input.FileName = "Indicator_LW_USGS_Bedrock.pfb"

model.GeomInput.s1.Value =    1
model.GeomInput.s2.Value =    2
model.GeomInput.s3.Value =    3
model.GeomInput.s4.Value =    4
model.GeomInput.s5.Value =    5
model.GeomInput.s6.Value =    6
model.GeomInput.s7.Value =    7
model.GeomInput.s8.Value =    8
model.GeomInput.s9.Value =    9
model.GeomInput.s10.Value =   10
model.GeomInput.s11.Value =   11
model.GeomInput.s12.Value =   12
model.GeomInput.s13.Value =   13

model.GeomInput.g1.Value =    19
model.GeomInput.g2.Value =    20
model.GeomInput.g3.Value =    21
model.GeomInput.g4.Value =    22
model.GeomInput.g5.Value =    23
model.GeomInput.g6.Value =    24
model.GeomInput.g7.Value =    25
model.GeomInput.g8.Value =    26
model.GeomInput.g9.Value =    27
model.GeomInput.g10.Value =    28

### 3.3 Variable ${\Delta z}$ 

In [ ]:
model.Solver.Nonlinear.VariableDz = True
model.dzScale.GeomNames = "domain"
model.dzScale.Type = "nzList"
model.dzScale.nzListNumber = 10

model.Cell._0.dzScale.Value = 5
model.Cell._1.dzScale.Value = 0.5
model.Cell._2.dzScale.Value = 0.25
model.Cell._3.dzScale.Value = 0.125
model.Cell._4.dzScale.Value = 0.05
model.Cell._5.dzScale.Value = 0.025
model.Cell._6.dzScale.Value = 0.005
model.Cell._7.dzScale.Value = 0.003
model.Cell._8.dzScale.Value = 0.0015
model.Cell._9.dzScale.Value = 0.0005

### 3.4 Topographic slopes 

In [ ]:
model.TopoSlopesX.Type = "PFBFile"
model.TopoSlopesX.GeomNames = "domain"
model.TopoSlopesX.FileName = "slopex_LW.pfb"

model.TopoSlopesY.Type = "PFBFile"
model.TopoSlopesY.GeomNames = "domain"
model.TopoSlopesY.FileName = "slopey_LW.pfb"

## 4. Setup the surface and subsurface properties 

### 4.1 Setting parameters using a csv table
Table readers can assign attribute data in a table format to parflow keys in python. Here we employ the table reader to assign our subsurface properties. Using this table will create the same keys in our ParFlow database as when we set them explicitly, but it is being done behind the scenes.  The `geom_table.csv` file we are using for this example has columns for: `permeability`, `porosity`,`relative permeability` and `saturation` properties.  

NOTE 1: The column headers in the input csv are case sensitive and  the key names must be the same as the GeomNames created above. 

NOTE 2: Its okay if you don't have values for every geometry for every variable (i.e. if some elements in your csv are blank). For anything you want to leave blank you need to have a `-` in that cell of the table as shown in this example. 

In [ ]:
geom_csv = os.path.join(run_dir, 'geom_table.csv')
geom_builder = SubsurfacePropertiesBuilder(model).load_csv_file(geom_csv)
geom_builder.apply(name_registration=True)

### 4.2 Setting the remaining parameters not set in the table

In [ ]:
# Permeability tensor
model.Perm.TensorType = "TensorByGeom"
model.Geom.Perm.TensorByGeom.Names = "domain"
model.Geom.domain.Perm.TensorValX = 1.0
model.Geom.domain.Perm.TensorValY = 1.0
model.Geom.domain.Perm.TensorValZ = 1.0

# Speicific storage
model.SpecificStorage.Type = "Constant"
model.SpecificStorage.GeomNames = "domain"
model.Geom.domain.SpecificStorage.Value = 0.0001

#Manning's coefficient
model.Mannings.Type = "Constant"
model.Mannings.GeomNames = "domain"
model.Mannings.Geom.domain.Value = 0.0000044

## 5. Phases contaminants, gravity and wells

In [ ]:
# Phases
model.Phase.Names = "water"
model.Phase.water.Density.Type = "Constant"
model.Phase.water.Density.Value = 1.0
model.Phase.water.Viscosity.Type = "Constant"
model.Phase.water.Viscosity.Value = 1.0
model.Phase.water.Mobility.Type = "Constant"
model.Phase.water.Mobility.Value = 1.0

# Contaminants
model.Contaminants.Names = ""

# Gravity
model.Gravity = 1.0

#Wells
model.Wells.Names = ""

# Phase Sources
model.PhaseSources.water.Type = "Constant"
model.PhaseSources.water.GeomNames = "domain"
model.PhaseSources.water.Geom.domain.Value = 0.0

## 6. Timing

In [ ]:
model.TimingInfo.BaseUnit = 1.0
model.TimingInfo.StartCount = 0
model.TimingInfo.StartTime = 0
model.TimingInfo.StopTime = 72.0
model.TimingInfo.DumpInterval = 1.0
model.TimeStep.Type = "Constant"
model.TimeStep.Value = 1.0

#Time cycles
model.Cycle.Names ="constant"
model.Cycle.constant.Names = "alltime"
model.Cycle.constant.alltime.Length = 1
model.Cycle.constant.Repeat = -1

# An alternate approach defining rainfall and recession time periods are defined here
# rain for 1 hour, recession for 2 hours
# model.Cycle.Names ="constant rainrec"
# model.Cycle.rainrec.Names = "rain rec"
# model.Cycle.rainrec.rain.Length = 1
# model.Cycle.rainrec.rec.Length = 5000000
# model.Cycle.rainrec.Repeat = -1

## 7. Boundary and intial conditions

### 7.1 Boundary conditions

In [ ]:
model.BCPressure.PatchNames = "top bottom side"

model.Patch.bottom.BCPressure.Type = "FluxConst"
model.Patch.bottom.BCPressure.Cycle = "constant"
model.Patch.bottom.BCPressure.alltime.Value = 0.0

model.Patch.side.BCPressure.Type = "FluxConst"
model.Patch.side.BCPressure.Cycle = "constant"
model.Patch.side.BCPressure.alltime.Value = 0.0

model.Patch.top.BCPressure.Type = "OverlandKinematic"
model.Patch.top.BCPressure.Cycle = "constant"
model.Patch.top.BCPressure.alltime.Value = 0.0

### 7.2 Initial conditions: water pressure

In [ ]:
model.ICPressure.Type = "PFBFile"
model.ICPressure.GeomNames = "domain"
model.Geom.domain.ICPressure.RefPatch = "top"
model.Geom.domain.ICPressure.FileName = "press.init.pfb"

## 8. CLM Settings

In [ ]:
model.Solver.LSM = "CLM"

# outputs
model.Solver.CLM.CLMFileDir = "clm_output/"
model.Solver.CLM.Print1dOut = False
model.Solver.BinaryOutDir = False #Solver: Field BinaryOutDir is not part of the expected schema <class 'parflow.tools.database.generated.Solver'>
model.Solver.CLM.DailyRST = True
model.Solver.CLM.CLMDumpInterval = 1

# forcing files
model.Solver.CLM.MetFileName = "NLDAS"
model.Solver.CLM.MetFilePath = "../../NLDAS"
model.Solver.CLM.MetForcing = "3D"
model.Solver.CLM.MetFileNT = 24
model.Solver.CLM.IstepStart = 1

# physical properties
model.Solver.CLM.EvapBeta = "Linear"
model.Solver.CLM.VegWaterStress = "Saturation"
model.Solver.CLM.ResSat = 0.1
model.Solver.CLM.WiltingPoint = 0.12
model.Solver.CLM.FieldCapacity = 0.98
model.Solver.CLM.IrrigationType = "none"
model.Solver.CLM.RootZoneNZ = 4
model.Solver.CLM.SoiLayer = 4


## 9. Solver Options

### 9.1 Output control

In [ ]:
model.Solver.PrintSubsurfData = True
model.Solver.PrintPressure = True
model.Solver.PrintSaturation = False
model.Solver.PrintMask = True
model.Solver.PrintVelocities = False
model.Solver.PrintEvapTrans = False
model.Solver.CLM.SingleFile = True

model.Solver.WriteCLMBinary = False
model.Solver.PrintCLM = True

### 9.2 Solver settings

In [ ]:
# Solver types
model.Solver = "Richards"
model.Solver.TerrainFollowingGrid = True
model.Solver.Linear.Preconditioner = "PFMG"
#model.Solver.Linear.Preconditioner.PCMatrixType = "FullJacobian"

# Exact solution
model.KnownSolution = "NoKnownSolution"

# Solver settings
model.Solver.MaxIter = 25000
model.Solver.Drop = 1e-20
model.Solver.AbsTol = 1e-8
model.Solver.MaxConvergenceFailures = 8
model.Solver.Nonlinear.MaxIter = 1000
model.Solver.Nonlinear.ResidualTol = 1e-6
model.Solver.Nonlinear.EtaChoice =  "EtaConstant"
model.Solver.Nonlinear.EtaValue = 0.001
model.Solver.Nonlinear.UseJacobian = True
model.Solver.Nonlinear.DerivativeEpsilon = 1e-16
model.Solver.Nonlinear.StepTol = 1e-15
model.Solver.Nonlinear.Globalization = "LineSearch"
model.Solver.Linear.KrylovDimension = 70
model.Solver.Linear.MaxRestarts = 2


## 10.  Distribute files, write the model and run the simulation

In [ ]:
# Distribute input files
# slope files are 2D (i.e. they only have one layer) so you need to set NZ to 1 before you distribute them
# Make sure to set it back to your actual NZ before distributing 3D files or running your model
model.ComputationalGrid.NZ =1
model.dist("slopex_LW.pfb")
model.dist("slopey_LW.pfb")

model.ComputationalGrid.NZ = 10 #the rest of the inputs shoul dbe distributed over 3D space
model.dist("Indicator_LW_USGS_Bedrock.pfb")
model.dist("press.init.pfb")

# write
model.write()
model.write(file_format='yaml')
model.write(file_format='json')

#run
model.run()